In [2]:
# Librerias
import pandas as pd # Guardar informacion en dataframes
import glob # Buscar un patron de un archivo
import os

In [7]:
# Cargar el archivo CSV con manejo de errores
def cargar_csv(ruta):
    try:
        df = pd.read_csv(ruta)
        return df
    except FileNotFoundError:
        print(f"Error: El archivo {ruta} no se encontró.")
        return None
    except pd.errors.ParserError:
        print("Error: Hubo un problema al analizar el archivo CSV.")
        return None

# Guardar genes agrupados por pathways en un solo archivo
def guardar_genes_por_pathway(df, salida):
    if df is not None:
        df_filtrado = df[['Pathway', 'Genes']]
        
        try:
            # Leer el archivo existente solo una vez al principio
            df_existente = pd.read_csv(salida)
            # Eliminar duplicados antes de agregar nuevas filas
            df_completo = pd.concat([df_existente, df_filtrado], ignore_index=True).drop_duplicates(subset=['Pathway', 'Genes'])
        except FileNotFoundError:
            # Si no existe, crear uno nuevo con la información filtrada
            df_completo = df_filtrado
        
        df_completo.to_csv(salida, index=False)
    else:
        print("No se pudo guardar el archivo porque hubo un error al cargar los datos.")

# Cargar y procesar los archivos CSV
rutas_csv = [
    "ShinyGo/Hospedero/enrichment_all.csv",  # Segundo archivo
]

salida_csv = "ShinyGo/Hospedero/genes_pathway_Human.csv"  # Archivo de salida único

# Procesar ambos archivos
for ruta in rutas_csv:
    df = cargar_csv(ruta)
    guardar_genes_por_pathway(df, salida_csv)

In [8]:
# Definir ruta de entrada y salida
input_file = "ShinyGo/Hospedero/genes_pathway_Human.csv"
output_dir = "ShinyGo/Hospedero/Pathway/"

# Crear el directorio de salida si no existe
os.makedirs(output_dir, exist_ok=True)

# Cargar el archivo CSV
df = pd.read_csv(input_file)

# Función para limpiar los nombres de los pathways
def clean_pathway_name(name):
    return name.replace(" ", "_").replace("/", "_").replace("\\", "_").replace(":", "_").replace("-", "_")

# Separar los genes en filas y guardar en archivos TSV
for index, row in df.iterrows():
    input_genes = row['Genes'].split()
    
    # Crear un DataFrame con la columna Genes
    df_genes = pd.DataFrame({'Genes': input_genes})
    
    # Generar el nombre del archivo
    pathway_name = clean_pathway_name(row['Pathway'])
    output_file = os.path.join(output_dir, f"{pathway_name}.tsv")
    
    # Guardar el DataFrame en un archivo TSV
    df_genes.to_csv(output_file, sep='\t', index=False)

In [9]:
# Definir los patrones de los archivos que queremos seleccionar
patterns = [
    "ShinyGo/Hospedero/Pathway/Path_*_Chagas_disease.tsv",
    "ShinyGo/Hospedero/Pathway/Path_*_Leukocyte_transendothelial_migration.tsv",
    "ShinyGo/Hospedero/Pathway/Path_*_Fc_gamma_R_mediatedphagocytosis.tsv",
    "ShinyGo/Hospedero/Pathway/Path_*_Viral_protein_interaction_with_cytokine_and_cytokine_receptor.tsv",
    "ShinyGo/Hospedero/Pathway/Path_*_Toll_like_receptor_signaling_pathway.tsv"
]

# Obtener la lista de archivos que coinciden con los patrones
input_files = [file for pattern in patterns for file in glob.glob(pattern)]

# Verificar si hay archivos antes de concatenar
if input_files:
    df_combined = pd.concat((pd.read_csv(file, sep="\t", header=None) for file in input_files), ignore_index=True)
    df_combined.drop_duplicates().to_csv("ShinyGo/HospederoBase_Human.tsv", sep="\t", index=False, header=False)
else:
    print("No se encontraron archivos con los patrones especificados.")

In [10]:
# Leer el archivo .tsv
tsv_file = "ShinyGo/HospederoBase_Human.tsv"
df_tsv = pd.read_csv(tsv_file, sep='\t')

# Extraer la primera columna, convertirla a un set para eliminar duplicados
protein_set = set(df_tsv.iloc[:, 0])

# Guardar el resultado sin nombre de columna
protein_df = pd.DataFrame(list(protein_set))
protein_df.to_csv("ShinyGo/HospederoBase_Human_SinProteinID.tsv", sep='\t', index=False, header=False)

In [12]:
# Cargar el archivo CSV con manejo de errores
def cargar_csv(ruta):
    try:
        df = pd.read_csv(ruta, sep='\t', header=None)  # Leer archivos .tsv
        return df
    except FileNotFoundError:
        print(f"Error: El archivo {ruta} no se encontró.")
        return None
    except pd.errors.ParserError:
        print("Error: Hubo un problema al analizar el archivo.")
        return None

# Guardar los archivos combinados en un solo archivo
def guardar_archivo_combinado(df1, df2, salida):
    if df1 is not None and df2 is not None:
        # Concatenar las dos columnas en un solo DataFrame
        df_combinado = pd.concat([df1, df2], ignore_index=True)
        df_combinado.to_csv(salida, index=False, header=False, sep='\t')
    else:
        print("No se pudo guardar el archivo porque hubo un error al cargar los datos.")

# Rutas de los archivos de entrada
ruta_HostDB = "ShinyGo/HospederoBase_Human_SinProteinID.tsv"
ruta_ChagasDB = "Host_DB/Chagas_metadata/Chagas_metadata_Protein.tsv"

# Ruta del archivo de salida
salida_tsv = "ShinyGo/HospederoBase_Human_SinProteinID2.tsv"

# Cargar los archivos
df_HostDB= cargar_csv(ruta_HostDB)
df_ruta_ChagasDB = cargar_csv(ruta_ChagasDB)

# Guardar el archivo combinado
guardar_archivo_combinado(df_HostDB, df_ruta_ChagasDB, salida_tsv)